In [9]:
#1.
import numpy as np

# Parameters
np.random.seed(0)  # For reproducibility
n_obs = 200
phi_0 = np.array([[0.2], [0.4]])
Phi = np.array([[0.2, 0.3], [-0.6, 1.1]])
Sigma = np.array([[2, 1], [1, 1]])

# Generate VAR(1) process
f_t = np.zeros((n_obs, 2))
a_t = np.random.multivariate_normal([0, 0], Sigma, size=n_obs)
f_t[0] = np.dot(Phi, phi_0).flatten() + a_t[0]

for t in range(1, n_obs):
    f_t[t] = np.dot(Phi, f_t[t - 1]) + a_t[t]

# Print the first few observations
print(f_t)

[[-2.39802871 -0.97021746]
 [-2.84589858  0.71712104]
 [-2.60697963  0.39394719]
 [-1.66171735  1.10976301]
 [ 0.00924246  2.52143733]
 [ 0.08749862  3.41006164]
 [-0.04649496  3.11515962]
 [ 0.2059062   3.25242291]
 [-0.97285535  2.07532383]
 [ 0.27463788  2.15123343]
 [ 4.00181307  4.71690452]
 [ 1.26678365  1.66199453]
 [-1.89954209 -1.62725009]
 [-0.87024492 -0.78758274]
 [-2.99743701 -0.87556632]
 [-1.19829658  0.9023449 ]
 [ 1.89657619  1.42538668]
 [ 1.23499037  0.8081289 ]
 [-1.59459    -0.2664717 ]
 [ 0.23247424  0.83418503]
 [ 2.2013515   0.92352382]
 [ 2.43196154  2.17208873]
 [ 1.98183394  1.13334743]
 [ 2.20807397  1.53202407]
 [ 3.19168534  1.62140141]
 [ 2.23154924  0.83366639]
 [ 1.78308337 -0.60807479]
 [ 0.07381029 -1.48957166]
 [-0.62194162 -1.58037904]
 [ 0.39242918 -1.01636969]
 [ 0.81596324 -0.97046342]
 [ 1.55215668 -1.77294463]
 [-0.33511165 -3.24368954]
 [ 1.05327927 -1.73696293]
 [ 0.92147796 -1.74352339]
 [-1.38417758 -3.02315365]
 [-2.35081288 -4.11338093]
 

In [6]:
#2.
# Parameters for VARMA(1,1) model
np.random.seed(0)
n_obs = 200
phi = np.array([[0.2, 0.3], [-0.6, 1.1]])
theta = np.array([[0.5, 0.4], [0.3, 0.2]])
mean = np.array([0, 0])
covariance = np.array([[2, 1], [1, 1]])

# Generate VARMA(1,1) data
f_t = np.zeros((n_obs, 2))
a_t = np.random.multivariate_normal(mean, covariance, size=n_obs)
for t in range(1, n_obs):
    f_t[t] = mean + np.dot(phi, f_t[t-1]) + np.dot(theta, a_t[t-1])

# Print the first few observations
print(f_t[:5])

[[ 0.          0.        ]
 [-1.79510134 -1.02545211]
 [-1.5660525  -0.60439619]
 [-2.46196751 -0.8215702 ]
 [-1.72322402  0.01834828]]


In [7]:
from statsmodels.tsa.api import VAR
import numpy as np

# Determine the maximum lag order to consider
max_lag = 10
T = len(f_t)

# Initialize an array to store M(i) values
m_values = []

# Fit VAR models with different orders
for i in range(1, max_lag + 1):
    model = VAR(f_t)
    result_i = model.fit(i)
    if i > 1:
        result_i_minus_1 = model.fit(i-1)
        sigma_i = result_i.sigma_u
        sigma_i_minus_1 = result_i_minus_1.sigma_u

        # Calculate the determinant of the covariance matrices
        det_sigma_i = np.linalg.det(sigma_i)
        det_sigma_i_minus_1 = np.linalg.det(sigma_i_minus_1)

        # Calculate M(i) statistic
        m_i = -(T - i - 3/2) * np.log(det_sigma_i / det_sigma_i_minus_1)
        m_values.append((i, m_i))

# Find the best model order based on M(i) statistics
best_order = sorted(m_values, key=lambda x: x[1])[0][0]

print(f"The estimated order of the VARMA model using M(i) statistics is {best_order}.")

The estimated order of the VARMA model using M(i) statistics is 7.


In [8]:
from statsmodels.tsa.api import VAR

# Fit a VAR model with the optimal order
model = VAR(f_t)
result = model.fit(7)

# Print the model summary to check the coefficients and statistics
print(result.summary())

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Thu, 23, May, 2024
Time:                     03:07:21
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                   -7.22876
Nobs:                     193.000    HQIC:                  -7.53054
Log likelihood:           228.806    FPE:                0.000437125
AIC:                     -7.73592    Det(Omega_mle):     0.000376352
--------------------------------------------------------------------
Results for equation y1
           coefficient       std. error           t-stat            prob
------------------------------------------------------------------------
const         0.124310         0.079458            1.564           0.118
L1.y1         1.133488         2.317164            0.489           0.625
L1.y2        -1.214898         4.101542           -0.296           0.767
L2.y1        -3.357127